In [242]:
!pip install --user --upgrade pip

     ---------------------------------------- 2.1/2.1 MB 131.6 kB/s eta 0:00:00


ERROR: To modify pip, please run the following command:
C:\Users\LENO\anaconda\python.exe -m pip install --user --upgrade pip


In [245]:
!pip install tensorflow==2.5.1 tensorflow-gpu==2.5.1 --user opencv-python mediapipe sklearn matplotlib


  Using cached tensorflow_gpu-2.5.1-cp39-cp39-win_amd64.whl (422.7 MB)


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


  Using cached tensorflow_gpu-2.5.1-cp39-cp39-win_amd64.whl (422.7 MB)


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [246]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [247]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [248]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [249]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [394]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [395]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)

        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break 
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [396]:
draw_landmarks(frame, results)

In [398]:
len(results.left_hand_landmarks.landmark)


21

In [399]:
pose = []
for res in results.pose_landmarks.landmark:
    test = np.array([res.x, res.y, res.z, res.visibility])
    pose.append(test)


In [400]:
pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

In [401]:
face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)

In [402]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [403]:
result_test = extract_keypoints(results)


In [404]:
result_test


array([ 0.58166754,  0.6433624 , -1.10780013, ...,  0.        ,
        0.        ,  0.        ])

In [405]:
np.save('0', result_test)


In [406]:
np.load('0.npy')

array([ 0.58166754,  0.6433624 , -1.10780013, ...,  0.        ,
        0.        ,  0.        ])

In [407]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

# Actions that we try to detect
actions = np.array(['My brother', 'and Myself', 'are Twins'])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

# Folder start
start_folder = 30

In [408]:
for action in actions: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

In [409]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)
#                 print(results)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(2000)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break 
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

In [410]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [411]:
label_map = {label:num for num, label in enumerate(actions)}


In [412]:
label_map


{'My brother': 0, 'and Myself': 1, 'are Twins': 2}

In [413]:
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [414]:
np.array(sequences).shape


(90, 30, 1662)

In [415]:
np.array(labels).shape


(90,)

In [416]:
X = np.array(sequences)


In [417]:
X.shape


(90, 30, 1662)

In [418]:
y = to_categorical(labels).astype(int)


In [419]:
y


array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0,

In [420]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)


In [421]:
y_test.shape

(5, 3)

In [422]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [423]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [424]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [425]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])


In [426]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])


Epoch 1/2000
3/3 [==============================] - 3s 425ms/step - loss: 2.5273 - categorical_accuracy: 0.3294
Epoch 2/2000
3/3 [==============================] - 0s 133ms/step - loss: 2.8569 - categorical_accuracy: 0.2941
Epoch 3/2000
3/3 [==============================] - 0s 136ms/step - loss: 1.4525 - categorical_accuracy: 0.2588
Epoch 4/2000
3/3 [==============================] - 0s 126ms/step - loss: 1.0966 - categorical_accuracy: 0.4118
Epoch 5/2000
3/3 [==============================] - 0s 126ms/step - loss: 1.0954 - categorical_accuracy: 0.3176
Epoch 6/2000
3/3 [==============================] - 0s 125ms/step - loss: 1.0936 - categorical_accuracy: 0.3176
Epoch 7/2000
3/3 [==============================] - 0s 127ms/step - loss: 1.0925 - categorical_accuracy: 0.2824
Epoch 8/2000
3/3 [==============================] - 0s 125ms/step - loss: 1.0894 - categorical_accuracy: 0.5529
Epoch 9/2000
3/3 [==============================] - 0s 128ms/step - loss: 1.0786 - categorical_accuracy:

3/3 [==============================] - 0s 153ms/step - loss: 1.0952 - categorical_accuracy: 0.3529
Epoch 146/2000
3/3 [==============================] - 0s 146ms/step - loss: 1.0952 - categorical_accuracy: 0.3529
Epoch 147/2000
3/3 [==============================] - 0s 151ms/step - loss: 1.0951 - categorical_accuracy: 0.3529
Epoch 148/2000
3/3 [==============================] - 0s 157ms/step - loss: 1.0951 - categorical_accuracy: 0.3529
Epoch 149/2000
3/3 [==============================] - 1s 166ms/step - loss: 1.0949 - categorical_accuracy: 0.3529
Epoch 150/2000
3/3 [==============================] - 1s 205ms/step - loss: 1.0948 - categorical_accuracy: 0.3529
Epoch 151/2000
3/3 [==============================] - 1s 164ms/step - loss: 1.0948 - categorical_accuracy: 0.3529
Epoch 152/2000
3/3 [==============================] - 1s 163ms/step - loss: 1.0946 - categorical_accuracy: 0.3529
Epoch 153/2000
3/3 [==============================] - 1s 168ms/step - loss: 1.0945 - categorical_accura

Epoch 217/2000
3/3 [==============================] - 1s 167ms/step - loss: 1.0950 - categorical_accuracy: 0.3529
Epoch 218/2000
3/3 [==============================] - 1s 172ms/step - loss: 1.0948 - categorical_accuracy: 0.3529
Epoch 219/2000
3/3 [==============================] - 1s 197ms/step - loss: 1.0944 - categorical_accuracy: 0.3529
Epoch 220/2000
3/3 [==============================] - 1s 183ms/step - loss: 1.0943 - categorical_accuracy: 0.3529
Epoch 221/2000
3/3 [==============================] - 1s 184ms/step - loss: 1.0935 - categorical_accuracy: 0.3529
Epoch 222/2000
3/3 [==============================] - 1s 168ms/step - loss: 1.0934 - categorical_accuracy: 0.3529
Epoch 223/2000
3/3 [==============================] - 0s 158ms/step - loss: 1.0930 - categorical_accuracy: 0.3529
Epoch 224/2000
3/3 [==============================] - 0s 157ms/step - loss: 1.0926 - categorical_accuracy: 0.3529
Epoch 225/2000
3/3 [==============================] - 0s 157ms/step - loss: 1.0922 - cat

3/3 [==============================] - 0s 158ms/step - loss: 0.8734 - categorical_accuracy: 0.5294
Epoch 360/2000
3/3 [==============================] - 0s 158ms/step - loss: 0.8658 - categorical_accuracy: 0.6235
Epoch 361/2000
3/3 [==============================] - 1s 160ms/step - loss: 0.8529 - categorical_accuracy: 0.5412
Epoch 362/2000
3/3 [==============================] - 0s 159ms/step - loss: 0.8462 - categorical_accuracy: 0.5176
Epoch 363/2000
3/3 [==============================] - 1s 161ms/step - loss: 0.8419 - categorical_accuracy: 0.7882
Epoch 364/2000
3/3 [==============================] - 1s 165ms/step - loss: 0.8373 - categorical_accuracy: 0.6118
Epoch 365/2000
3/3 [==============================] - 1s 165ms/step - loss: 0.8175 - categorical_accuracy: 0.7294
Epoch 366/2000
3/3 [==============================] - 1s 163ms/step - loss: 0.8118 - categorical_accuracy: 0.7412
Epoch 367/2000
3/3 [==============================] - 1s 160ms/step - loss: 0.7985 - categorical_accura

Epoch 431/2000
3/3 [==============================] - 1s 193ms/step - loss: 0.5284 - categorical_accuracy: 0.9647
Epoch 432/2000
3/3 [==============================] - 1s 195ms/step - loss: 0.5244 - categorical_accuracy: 0.9412
Epoch 433/2000
3/3 [==============================] - 1s 193ms/step - loss: 0.5236 - categorical_accuracy: 0.9412
Epoch 434/2000
3/3 [==============================] - 1s 163ms/step - loss: 0.5224 - categorical_accuracy: 0.9529
Epoch 435/2000
3/3 [==============================] - 1s 159ms/step - loss: 0.5206 - categorical_accuracy: 0.9412
Epoch 436/2000
3/3 [==============================] - 1s 165ms/step - loss: 0.5197 - categorical_accuracy: 0.9412
Epoch 437/2000
3/3 [==============================] - 1s 175ms/step - loss: 0.5250 - categorical_accuracy: 0.9412
Epoch 438/2000
3/3 [==============================] - 1s 174ms/step - loss: 0.5148 - categorical_accuracy: 0.9176
Epoch 439/2000
3/3 [==============================] - 1s 173ms/step - loss: 0.5116 - cat

...

3/3 [==============================] - 1s 179ms/step - loss: 1.8083 - categorical_accuracy: 0.3647
Epoch 1429/2000
3/3 [==============================] - 1s 185ms/step - loss: 2.2314 - categorical_accuracy: 0.3529
Epoch 1430/2000
3/3 [==============================] - 1s 181ms/step - loss: 1.7462 - categorical_accuracy: 0.4588
Epoch 1431/2000
3/3 [==============================] - 1s 173ms/step - loss: 1.6672 - categorical_accuracy: 0.3412
Epoch 1432/2000
3/3 [==============================] - 1s 174ms/step - loss: 1.9903 - categorical_accuracy: 0.3294
Epoch 1433/2000
3/3 [==============================] - 1s 162ms/step - loss: 2.5238 - categorical_accuracy: 0.3529
Epoch 1434/2000
3/3 [==============================] - 1s 163ms/step - loss: 2.2519 - categorical_accuracy: 0.3412
Epoch 1435/2000
3/3 [==============================] - 1s 161ms/step - loss: 2.6743 - categorical_accuracy: 0.3059
Epoch 1436/2000
3/3 [==============================] - 1s 159ms/step - loss: 4.3485 - categorica

3/3 [==============================] - 1s 165ms/step - loss: 1.5969 - categorical_accuracy: 0.4824
Epoch 1571/2000
3/3 [==============================] - 1s 192ms/step - loss: 1.7784 - categorical_accuracy: 0.4235
Epoch 1572/2000
3/3 [==============================] - 1s 192ms/step - loss: 1.5465 - categorical_accuracy: 0.4353
Epoch 1573/2000
3/3 [==============================] - 1s 199ms/step - loss: 1.2205 - categorical_accuracy: 0.4588
Epoch 1574/2000
3/3 [==============================] - 1s 200ms/step - loss: 1.3364 - categorical_accuracy: 0.5529
Epoch 1575/2000
3/3 [==============================] - 1s 162ms/step - loss: 1.5525 - categorical_accuracy: 0.3294
Epoch 1576/2000
3/3 [==============================] - 1s 162ms/step - loss: 1.3928 - categorical_accuracy: 0.3882
Epoch 1577/2000
3/3 [==============================] - 1s 159ms/step - loss: 2.0285 - categorical_accuracy: 0.3412
Epoch 1578/2000
3/3 [==============================] - 0s 158ms/step - loss: 1.6341 - categorica

3/3 [==============================] - 0s 160ms/step - loss: 1.3999 - categorical_accuracy: 0.6000
Epoch 1713/2000
3/3 [==============================] - 1s 167ms/step - loss: 2.3211 - categorical_accuracy: 0.3412
Epoch 1714/2000
3/3 [==============================] - 1s 162ms/step - loss: 2.0422 - categorical_accuracy: 0.4824
Epoch 1715/2000
3/3 [==============================] - 1s 161ms/step - loss: 2.9870 - categorical_accuracy: 0.3647
Epoch 1716/2000
3/3 [==============================] - 0s 158ms/step - loss: 1.4677 - categorical_accuracy: 0.5529
Epoch 1717/2000
3/3 [==============================] - 0s 158ms/step - loss: 2.4944 - categorical_accuracy: 0.2941
Epoch 1718/2000
3/3 [==============================] - 1s 184ms/step - loss: 2.1806 - categorical_accuracy: 0.4941
Epoch 1719/2000
3/3 [==============================] - 1s 185ms/step - loss: 2.5629 - categorical_accuracy: 0.4000
Epoch 1720/2000
3/3 [==============================] - 1s 188ms/step - loss: 1.5804 - categorica

3/3 [==============================] - 0s 157ms/step - loss: 0.4249 - categorical_accuracy: 0.8353
Epoch 1855/2000
3/3 [==============================] - 0s 156ms/step - loss: 0.4016 - categorical_accuracy: 0.8588
Epoch 1856/2000
3/3 [==============================] - 1s 176ms/step - loss: 0.5165 - categorical_accuracy: 0.7765
Epoch 1857/2000
3/3 [==============================] - 1s 170ms/step - loss: 0.7502 - categorical_accuracy: 0.6353
Epoch 1858/2000
3/3 [==============================] - 1s 171ms/step - loss: 0.6246 - categorical_accuracy: 0.7412
Epoch 1859/2000
3/3 [==============================] - 0s 159ms/step - loss: 0.4900 - categorical_accuracy: 0.7647
Epoch 1860/2000
3/3 [==============================] - 0s 159ms/step - loss: 0.3560 - categorical_accuracy: 0.8941
Epoch 1861/2000
3/3 [==============================] - 0s 156ms/step - loss: 0.3961 - categorical_accuracy: 0.8706
Epoch 1862/2000
3/3 [==============================] - 0s 159ms/step - loss: 0.3488 - categorica

3/3 [==============================] - 1s 167ms/step - loss: 1.0384 - categorical_accuracy: 0.5882
Epoch 1997/2000
3/3 [==============================] - 1s 159ms/step - loss: 1.0693 - categorical_accuracy: 0.6000
Epoch 1998/2000
3/3 [==============================] - 0s 157ms/step - loss: 0.6272 - categorical_accuracy: 0.8118
Epoch 1999/2000
3/3 [==============================] - 1s 160ms/step - loss: 0.7770 - categorical_accuracy: 0.6941
Epoch 2000/2000
3/3 [==============================] - 1s 162ms/step - loss: 1.2646 - categorical_accuracy: 0.6941


In [427]:
model.summary()


Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_18 (LSTM)               (None, 30, 64)            442112    
_________________________________________________________________
lstm_19 (LSTM)               (None, 30, 128)           98816     
_________________________________________________________________
lstm_20 (LSTM)               (None, 64)                49408     
_________________________________________________________________
dense_18 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_19 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_20 (Dense)             (None, 3)                 99        
Total params: 596,675
Trainable params: 596,675
Non-trainable params: 0
________________________________________________

In [428]:
res = model.predict(X_test)


In [429]:
actions[np.argmax(res[4])]


'are Twins'

In [430]:
actions[np.argmax(y_test[4])]


'are Twins'

In [431]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score


In [432]:
yhat = model.predict(X_test)


In [433]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [434]:
multilabel_confusion_matrix(ytrue, yhat)


array([[[2, 0],
        [0, 3]],

       [[3, 0],
        [0, 2]]], dtype=int64)

In [435]:
accuracy_score(ytrue, yhat)


1.0

In [436]:
from scipy import stats


In [437]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [442]:
# 1. New detection variables
sequence = []
sentence = []
threshold = 0.7

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
#         sequence.insert(0,keypoints)
#         sequence = sequence[:30]
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            
            
        #3. Viz logic
            if res[np.argmax(res)] > threshold: 
                if len(sentence) > 0: 
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
and Myself
<class 'mediapipe.python.solution_base.SolutionOutputs'>
and Myself
<class 'mediapipe.python.solution_base.SolutionOutputs'>
and Myself
<class 'mediapipe.python.solution_base.SolutionOutputs'>
and Myself
<class 'mediapipe.python.solution_base.SolutionOutputs'>
and Myself
<class 'mediapipe.python.solution_base.SolutionOutputs'>
and Myself
<class 'mediapipe.python.solution_base.SolutionOutputs'>
and Myself
<class 'mediapipe.python.solution_base.SolutionOutputs'>
and Myself
<class 'mediapipe.python.solution_base.SolutionOutputs'>
and Myself
<class 'mediapipe.python.solution_base.SolutionOutputs'>
and Myself
<class 'mediapipe.python.solution_base.SolutionOutputs'>
and Myself
<class 'mediapipe.python.solution_base.SolutionOutputs'>
and Myself
<class 'mediapipe.python.solution_base.SolutionOutputs'>
and Myself
<class 'mediapipe.python.solution_base.SolutionOutputs'>
My brother
<class 'mediapipe.python.solution_base.SolutionO

<class 'mediapipe.python.solution_base.SolutionOutputs'>
My brother
<class 'mediapipe.python.solution_base.SolutionOutputs'>
My brother
<class 'mediapipe.python.solution_base.SolutionOutputs'>
My brother
<class 'mediapipe.python.solution_base.SolutionOutputs'>
My brother
<class 'mediapipe.python.solution_base.SolutionOutputs'>
My brother
<class 'mediapipe.python.solution_base.SolutionOutputs'>
My brother
<class 'mediapipe.python.solution_base.SolutionOutputs'>
My brother
<class 'mediapipe.python.solution_base.SolutionOutputs'>
My brother
<class 'mediapipe.python.solution_base.SolutionOutputs'>
My brother
<class 'mediapipe.python.solution_base.SolutionOutputs'>
My brother
<class 'mediapipe.python.solution_base.SolutionOutputs'>
My brother
<class 'mediapipe.python.solution_base.SolutionOutputs'>
My brother
<class 'mediapipe.python.solution_base.SolutionOutputs'>
My brother
<class 'mediapipe.python.solution_base.SolutionOutputs'>
My brother
<class 'mediapipe.python.solution_base.SolutionO